In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import datetime as dt
from autogen import AssistantAgent
from inswitch.llm.model import get_openai_model_config
import time
import requests

app = Flask(__name__)
CORS(app)

llm_config = {"config_list": [get_openai_model_config()]}
assistant = AssistantAgent(name="assistant", llm_config=llm_config)

In [ ]:
# REPOSITORY
# Handles access to the database

def get_next_intent_id():
    sparql_query = """
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX ex: <http://example.com/intent/>

    SELECT (MAX(xsd:integer(STRAFTER(STR(?intent), "http://example.com/intent/"))) AS ?maxId)
    WHERE {
    ?intent a icm:Intent .
    }
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data={"query": sparql_query},
        headers={"Accept": "application/sparql-results+json"}
    )

    # TODO return None and handle it
    if response.status_code != 200:
        return 1
    
    results = response.json()["results"]["bindings"]
    
    if not results or "maxId" not in results[0]:
        return 1

    max_id = int(results[0]["maxId"]["value"])
    return max_id + 1

def get_next_report_id(intent_id):
    sparql_query = f"""
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX ex: <http://example.com/intent/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT (MAX(xsd:integer(STRAFTER(STR(?report), "http://example.com/intent/{intent_id}/intentReport/"))) AS ?maxId)
    WHERE {{
        ?report a icm:IntentReport ;
                ex:intent ex:{intent_id} .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data={"query": sparql_query},
        headers={"Accept": "application/sparql-results+json"}
    )

    # TODO return None and handle it
    if response.status_code != 200:
        return 1
    
    results = response.json()["results"]["bindings"]
    
    if not results or "maxId" not in results[0]:
        return 1

    max_id = int(results[0]["maxId"]["value"])
    return max_id + 1

def fetch_intents_from_graphdb():
    sparql_query = """
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT ?id ?authorName ?content ?date ?lastUpdate
    WHERE {
        ?intent a icm:Intent ;
        ex:author ?author ;
        ex:content ?content ;
        dcterms:date ?date ;
        dcterms:lastUpdateDate ?lastUpdate .
        ?author foaf:name ?authorName .
        BIND(STR(?intent) AS ?id)
    }
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data={"query": sparql_query},
        headers={"Accept": "application/sparql-results+json"}
    )

    if response.status_code != 200:
        return None
    
    return response.json()["results"]["bindings"]


def insert_intent_into_graphdb(id, author, content, date):
    author_without_space = author.replace(" ", "_")

    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {{
        ex:{id} a icm:Intent ;
            ex:author <http://example.com/intent/person/{author_without_space}> ;
            ex:content "{content}" ;
            dcterms:date "{date}"^^xsd:dateTime ;
            dcterms:lastUpdateDate "{date}"^^xsd:dateTime .

        <http://example.com/intent/person/{author_without_space}> a foaf:Person ;
            foaf:name "{author}" .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query,
        headers={"Content-Type": "application/sparql-update"}
    )

    return response

def insert_report_into_graphdb(id, intent_id, agent_name, agent_response, response_time):
    date = dt.datetime.now().isoformat()

    # Cleaning the response
    agent_response = agent_response.replace('"', '\\"')

    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {{
        <http://example.com/intent/{intent_id}/intentReport/{id}> a icm:IntentReport ;
            dcterms:date "{date}"^^xsd:dateTime ;
            ex:agentName "{agent_name}" ;
            ex:executionTime "{response_time}"^^xsd:float ;
            ex:intent <http://example.com/intent/{intent_id}> ;
            ex:response "{agent_response}" .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )
    
    return response

def select_intent_from_graphdb(intent_id):
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT ?id ?authorName ?content ?date ?lastUpdate
    WHERE {{
        BIND(<http://example.com/intent/{intent_id}> AS ?intent)
        ?intent a icm:Intent ;
        ex:author ?author ;
        ex:content ?content ;
        dcterms:date ?date ;
        dcterms:lastUpdateDate ?lastUpdate .
        ?author foaf:name ?authorName .
        BIND(STR(?intent) AS ?id)
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data=sparql_query.encode("utf-8"),
        headers={
            "Content-Type": "application/sparql-query",
            "Accept": "application/sparql-results+json"
        }
    )
    
    return response.json()["results"]["bindings"]

def fetch_intent_reports_of_intent_from_graphdb(intent_id):
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>

    SELECT ?report ?date ?agentName ?response ?executionTime
    WHERE {{
        ?report a icm:IntentReport ;
                dcterms:date ?date ;
                ex:agentName ?agentName ;
                ex:response ?response ;
                ex:executionTime ?executionTime ;
                ex:intent <http://example.com/intent/{intent_id}> .
    }}
    ORDER BY DESC(?date)
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data=sparql_query.encode("utf-8"),
        headers={
            "Content-Type": "application/sparql-query",
            "Accept": "application/sparql-results+json"
        }
    )

    return response.json()["results"]["bindings"]

def fetch_single_intent_report_from_graphdb(intent_id, report_id):
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>

    SELECT ?report ?date ?agentName ?response ?executionTime
    WHERE {{
        BIND(<http://example.com/intent/{intent_id}/intentReport/{report_id}> AS ?report)
        ?report a icm:IntentReport ;
            dcterms:date ?date ;
            ex:agentName ?agentName ;
            ex:response ?response ;
            ex:executionTime ?executionTime ;
            ex:intent <http://example.com/intent/{intent_id}> .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data=sparql_query.encode("utf-8"),
        headers={
            "Content-Type": "application/sparql-query",
            "Accept": "application/sparql-results+json"
        }
    )

    return response.json()["results"]["bindings"]


def update_intent_content_from_graphdb(intent_id, new_content):
    updateDate = dt.datetime.now().isoformat()

    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    
    DELETE {{
        <http://example.com/intent/{intent_id}> ex:content ?oldContent .
        <http://example.com/intent/{intent_id}> dcterms:lastUpdateDate ?oldDate .
    }}
    INSERT {{
        <http://example.com/intent/{intent_id}> ex:content "{new_content}" .
        <http://example.com/intent/{intent_id}> dcterms:lastUpdateDate "{updateDate}"^^xsd:dateTime .
    }}
    WHERE {{
        <http://example.com/intent/{intent_id}> ex:content ?oldContent .
        OPTIONAL {{ <http://example.com/intent/{intent_id}> dcterms:lastUpdateDate ?oldDate . }}
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )

    return response


def delete_intent_from_graphdb(intent_id):
    sparql_query = f"""
    DELETE {{
        <http://example.com/intent/{intent_id}> ?p ?o .
    }}
    WHERE {{
        <http://example.com/intent/{intent_id}> ?p ?o .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )
    
    return response

def delete_reports_of_intent_from_graphdb(intent_id):
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>

    DELETE {{
        ?report ?p ?o .
    }}
    WHERE {{
        ?report ex:intent <http://example.com/intent/{intent_id}> .
        ?report ?p ?o .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )
    
    return response

def delete_report_of_intent_from_graphdb(intent_id, report_id):
    sparql_query = f"""
    DELETE {{
        <http://example.com/intent/{intent_id}/intentReport/{report_id}> ?p ?o .
    }}
    WHERE {{
        <http://example.com/intent/{intent_id}/intentReport/{report_id}> ?p ?o .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )

    return response


In [ ]:
# SERVICE
# Contains logic

def get_all_intents():
    results = fetch_intents_from_graphdb()

    if not results:
        return None

    intents = []

    for i in results:
        id = i['id']["value"].split("/")[-1]

        intent = {
            "id": id,
            "author": i["authorName"]["value"],
            "content": i["content"]["value"],
            "date": i["date"]["value"],
            "lastUpdateDate": i["lastUpdate"]["value"]
        }
        intents.append(intent)
    
    return intents

def get_all_intents_json_ld():
    intents = get_all_intents()
    
    if not intents:
        return None
    
    # Definition of the context
    jsonld_data = {
        "@context": {
            "ex": "http://example.com/intent/",
            "icm": "http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "dcterms": "http://purl.org/dc/terms/",
            "xsd": "http://www.w3.org/2001/XMLSchema# "
        }
    }
    jsonld_data["intents"] = []

    for intent in intents:
        id = intent['id'].split("/")[-1]
        
        date_obj = dt.datetime.fromisoformat(intent['date'])
        update_obj = dt.datetime.fromisoformat(intent['lastUpdateDate'])

        intent_data = {
            "@id": f"ex:{id}",
            "@type": "icm:Intent",
            "ex:author": {
                "@id": f"ex:person/{intent['author'].replace(' ', '_')}",
                "@type": "foaf:Person",
                "foaf:name": intent['author']
            },
            "ex:content": intent['content'],
            "dcterms:date": {
                "@value": date_obj.isoformat(),
                "@type": "xsd:dateTime"
            },
            "dcterms:lastUpdateDate": {
                "@value": update_obj.isoformat(),
                "@type": "xsd:dateTime"
            }
        }
        jsonld_data["intents"].append(intent_data)

    return jsonld_data

def select_intent_from_id(intent_id):
    result = select_intent_from_graphdb(intent_id)

    intent_db = result[0]

    if not intent_db:
        return None

    id = intent_db['id']["value"].split("/")[-1]

    intent = {
        "id": id,
        "author": intent_db["authorName"]["value"],
        "content": intent_db["content"]["value"],
        "date": intent_db["date"]["value"],
        "lastUpdateDate": intent_db["lastUpdate"]["value"]
    }
    
    return intent

def select_intent_json_ld_from_id(intent_id):
    result = select_intent_from_graphdb(intent_id)

    intent_db = result[0]

    if not intent_db:
        return None

    id = intent_db['id']["value"].split("/")[-1]
    
    date_obj = dt.datetime.fromisoformat(intent_db["date"]["value"])
    update_obj = dt.datetime.fromisoformat(intent_db["lastUpdate"]["value"])

    jsonld_data = {
        # Definition of the context
        "@context": {
            "ex": "http://example.com/intent/",
            "icm": "http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "dcterms": "http://purl.org/dc/terms/",
            "xsd": "http://www.w3.org/2001/XMLSchema# "
        },
        "@id": f"ex:{id}",
        "@type": "icm:Intent",
        "ex:author": {
            "@id": f"ex:person/{intent_db["authorName"]["value"].replace(' ', '_')}",
            "@type": "foaf:Person",
            "foaf:name": intent_db["authorName"]["value"]
        },
        "ex:content": intent_db["content"]["value"],
        "dcterms:date": {
            "@value": date_obj.isoformat(),
            "@type": "xsd:dateTime"
        },
        "dcterms:lastUpdateDate": {
            "@value": update_obj.isoformat(),
            "@type": "xsd:dateTime"
        }
    }
    
    return jsonld_data

def get_all_intent_reports_of_intent(intent_id):
    if(select_intent_from_id(intent_id)):
        results = fetch_intent_reports_of_intent_from_graphdb(intent_id)

        if not results:
            return None

        intent_reports = []

        for r in results:
            id = r['report']["value"].split("/")[-1]

            intent_report = {
                "id": id,
                "agentName": r["agentName"]["value"],
                "response": r["response"]["value"],
                "date": r["date"]["value"],
                "executionTime": r["executionTime"]["value"]
            }
            intent_reports.append(intent_report)
        
        return intent_reports
    else:
        return None
    
def get_intent_report_of_intent(intent_id, report_id):
    if select_intent_from_id(intent_id):
        result = fetch_single_intent_report_from_graphdb(intent_id, report_id)

        if not result:
            return None

        intent_report_data = result[0]

        id = result[0]['report']["value"].split("/")[-1]

        intent_report = {
            "id": id,
            "agentName": intent_report_data["agentName"]["value"],
            "response": intent_report_data["response"]["value"],
            "date": intent_report_data["date"]["value"],
            "executionTime": intent_report_data["executionTime"]["value"]
        }
        
        return intent_report
    else:
        return None


def create_intent(author, content):
    intent_id = get_next_intent_id()
    date = dt.datetime.now().isoformat()
    
    response = insert_intent_into_graphdb(intent_id, author, content, date)
    
    if response.status_code not in {200, 201, 202, 204}:
        return 
    
    # Handle of the intent

    startTime = time.time()
    
    # TODO
    """
    agent_response = assistant.generate_reply(
        messages=[{"role": "user", "content": content}]
    )
    """ 
    agent_response = "reply"

    # Cleaning of the response

    if(agent_response.endswith("TERMINATE")):
        agent_response = agent_response.replace("TERMINATE", "").strip()

    report_id = get_next_report_id(intent_id)
    print("Id report :", report_id)
    
    response = insert_report_into_graphdb(report_id, intent_id, assistant.name, agent_response, (time.time() - startTime))

    if response.status_code not in {200, 201, 202, 204}:
        return
    
    # TODO return intent and report
    # {"intent":intent.to_dict(), "report":report.to_dict()}
    return 201

def update_partially_intent(intent_id, content):
    if(select_intent_from_id(intent_id)):
        return update_intent_content_from_graphdb(intent_id, content)
    else:
        return None

def delete_intent_by_id(intent_id):
    response = delete_reports_of_intent_from_graphdb(intent_id)
    return delete_intent_from_graphdb(intent_id)

def delete_report_by_id(intent_id, report_id):
    return delete_report_of_intent_from_graphdb(intent_id, report_id)
    


In [ ]:
# CONTROLLER
# Handles HTTP requests

# GET METHODS

# Retrieves the list of intent objects

@app.get("/intent")
def get_intents():
    try:
        intents = get_all_intents()
        if intents:
            return jsonify(intents), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500

# Retrieves the list of intent and return it as RDF in JSON-LD format

@app.get("/intent/json-ld")
def get_intents_json_ld():
    try:
        intents = get_all_intents_json_ld()
        if intents:
            return jsonify(intents), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500
    
# Retrieves an Intent by ID

@app.get("/intent/<int:intent_id>")
def get_intent_from_id(intent_id):
    try:
        intent = select_intent_from_id(intent_id)
        if intent:
            return jsonify(intent), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500

# Retrieves an Intent by ID and return it as RDF in JSON-LD format

@app.get("/intent/<int:intent_id>/json-ld")
def get_intent_json_ld_from_id(intent_id):
    try:
        intent = select_intent_json_ld_from_id(intent_id)
        if intent:
            return jsonify(intent), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500
    

# Intent Report

# Retrieves the list of IntentReport of an Intent

@app.get("/intent/<int:intent_id>/intentReport")
def get_intent_report_of_intent_id(intent_id):
    try:
        intent_reports = get_all_intent_reports_of_intent(intent_id)
        if intent_reports:
            return jsonify(intent_reports), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500

# Retrieves an IntentReport of an Intent

@app.get("/intent/<int:intent_id>/intentReport/<int:report_id>")
def get_intent_report_from_id_of_intent_id(intent_id, report_id):
    try:
        intent_report = get_intent_report_of_intent(intent_id, report_id)
        if intent_report:
            return jsonify(intent_report), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500

# POST METHODS

# Creates an Intent

@app.post("/intent")
def add_intent():
    if request.is_json:
       added_intent = request.get_json()
       
       create_intent(added_intent["author"], added_intent["content"])

       return {"message": "Intent created"}, 201
    else :
        return {"error": "Request must be JSON"}, 415
    

# PATCH METHODS

# Partially updates an Intent

@app.patch("/intent/<int:intent_id>")
def update_intent(intent_id):
    updated_intent = request.get_json()

    if "content" in updated_intent and updated_intent["content"] != "":
        response = update_partially_intent(intent_id, updated_intent["content"])

        if(response):
            return get_intent_from_id(intent_id)
        else :
            return {"error": "No intent with id " + str(intent_id)}, 404
        

# DELETE METHODS

# Deletes an intent

@app.delete("/intent/<int:intent_id>")
def delete_intent(intent_id):
    # TODO check if intent exists
    response = delete_intent_by_id(intent_id)
    if response.status_code == 204:
        return jsonify({"message": "Intent deleted successfully"}), 200
    else:
        return jsonify({"error": response.text}), 500


# Deletes an IntentReport of an Intent

@app.delete("/intent/<int:intent_id>/intentReport/<int:report_id>")
def delete_intent_report_from_intent(intent_id, report_id):
    # TODO check if intent and report exist
    response = delete_report_by_id(intent_id, report_id)
    if response.status_code == 204:
        return jsonify({"message": "IntentReport deleted successfully"}), 200
    else:
        return jsonify({"error": response.text}), 500



In [ ]:
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)